In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(Seurat)
    library(Signac)
    library(EnsDb.Hsapiens.v86)
    library(dplyr)
    library(ggplot2)
    library(bedr)
    library(SeuratDisk)
    library(scales)
    library(reshape2)
    library(Hmisc)
    library(tidyr)
    library(tidyverse)
    library(crayon)
    library(readr)
}))

# Definitions etc

In [2]:
set.seed(1234)
options(digits=2)
stats <- c()

In [3]:
if(!exists("papermill_run")) {
    prj_name = "Screen1_66guides"
    data_subdir_suffixes = "L1,L2,L3,L4"     
    secondary_a_path = "/home/vsevim/prj/1012-ckd/S1/analysis/secondary/"
    save_seurat_h5 = "YES"
}

In [4]:
custom_theme <- theme(
  plot.title = element_text(size=16, hjust = 0.5), 
  legend.key.size = unit(0.7, "cm"), 
  legend.text = element_text(size = 14))

### Load Seurat file

In [5]:
integrated_h5_path <- "/home/vsevim/prj/1012-ckd/S1/analysis/secondary/integrated/seurat_objects/integrated.h5seurat"
seurat_combined    <- LoadH5Seurat(integrated_h5_path, verbose = FALSE)

Validating h5Seurat file

Warning message:
"Adding a command log without an assay associated with it"


Load THRESHOLDS.tsv

In [ ]:
thr_f_name = paste0(secondary_a_path, "/integrated/THRESHOLDS.tsv")

# Differential expression

Split construct+/- cells

In [7]:
table(seurat_combined$library)


Lib_1 Lib_2 Lib_3 Lib_4 
 6653  6936  5111  5863 

In [17]:
head(seurat_combined[['sgRNA']]@counts)

  [[ suppressing 34 column names 'AGCTCAATCGTAGGGA-1_1', 'CAACAGTTCCTAGCGG-1_1', 'AAACGCTCAGTTAGAA-1_1' ... ]]



6 x 24563 sparse Matrix of class "dgCMatrix"
                                                                           
CD13-1a . . . . . . . 1 . . 2 . . . . . . . . . . . . . . . . . . . . . . .
CD13-1b . . . . . . . 1 . . . . . . . . . . . . . . . . . . . 1 . . 1 . . .
CD55-1a . . . . . . . . . . . . . . . . . 1 . . . . . . . . . . . . . . . .
CD55-1b . 2 . . . . . 2 . 2 3 1 . . . . . 1 . 1 1 . . . . 2 . . . 1 . . 1 1
CD73-1a . . . . . . . 1 . . . . . . . . . . . . . . . . . . . . . . . . . .
CD73-1b . . . . . . . 1 . . . . . . . . . . . . 1 1 1 . . . 1 . . . 2 . . .
              
CD13-1a ......
CD13-1b ......
CD55-1a ......
CD55-1b ......
CD73-1a ......
CD73-1b ......

 .....suppressing 24529 columns in show(); maybe adjust 'options(max.print= *, width = *)'
 ..............................

In [ ]:
seurat_dummy <- seurat[['RNA']]